!%pip install llama-index llama-index-llms-ollama llama-index-embeddings-huggingface

In [15]:
import setup
setup.init_django()

In [16]:
django_pdf="data/django.pdf"

In [17]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.vector_stores.upstash import UpstashVectorStore

In [18]:
reader=SimpleDirectoryReader(input_files=[django_pdf])

In [5]:
data=reader.load_data()

In [37]:
from llama_index.llms.ollama import Ollama
llm = Ollama(model="phi3-mini", request_timeout=10.0)
Settings.llm=llm

In [38]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding


Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5",
    embed_batch_size=32
)

In [39]:
from llama_index.core.settings import Settings
Settings.context_window = 512

In [8]:
index = VectorStoreIndex.from_documents(data,
    insert_batch_size=500, show_progress=True)

Parsing nodes:   0%|          | 0/2888 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/500 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/500 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/500 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/500 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/500 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/396 [00:00<?, ?it/s]

In [40]:
query_engine = index.as_query_engine(
    similarity_top_k=2,  # Only use 2 most relevant chunks
)

In [41]:
# Save the index to disk to avoid rebuilding
index.storage_context.persist("./cache")

# Later, load from disk instead of rebuilding
from llama_index.core import StorageContext, load_index_from_storage
storage_context = StorageContext.from_defaults(persist_dir="./cache")
index = load_index_from_storage(storage_context)

In [42]:
from llama_index.core.response_synthesizers import get_response_synthesizer

summarizer = get_response_synthesizer(
    response_mode="tree_summarize",
    use_async=True
)

query_engine = index.as_query_engine(
    response_synthesizer=summarizer,
    similarity_top_k=2
)

In [43]:
r=query_engine.query("What are the arguments of embeddings field")
print(r.response)

ResponseError: model "phi3-mini" not found, try pulling it first (status code: 404)